In [1]:
import torch
import torch.nn as nn
from collections import deque
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from models.VGG import VGG
import matplotlib.pyplot as plt
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# local imports
from rails import RAILS
from aise import AISE
from pgd import PGD

In [2]:
%matplotlib inline

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


In [4]:
ROOT = "./datasets"
TRANSFORM = transforms.ToTensor()

train_data = datasets.CIFAR10(root=ROOT, 
                              download=True, 
                              train=True, 
                              transform=TRANSFORM)

test_data = datasets.CIFAR10(root=ROOT, 
                             download=True, 
                             train=False, 
                             transform=TRANSFORM)

# trainning data
x_train = (torch.FloatTensor(train_data.data[:50000])/255).permute(0,3,1,2)
y_train = torch.LongTensor(train_data.targets[:50000])


"""
baseline VGG model
"""

model = VGG()
model.load_state_dict(torch.load(
    "./model_weights/cifar_vgg16.pt", map_location=DEVICE
)['state_dict'])
model.eval()


"""
PGD attack
"""
pgd = PGD(eps=8/255,
          step=2/255,
          max_iter=10, 
          batch_size=256)


# evaluate the models and the attack
x_batch = (torch.FloatTensor(test_data.data[:1000])/255).permute(0,3,1,2)
y_batch = torch.LongTensor(test_data.targets[:1000])

x_adv = pgd.generate(model, x_batch, y_batch, device=DEVICE)

pred_clean = model(x_batch.to(DEVICE))[-1].max(dim=1)[1].detach().cpu()
clean_acc = (pred_clean == y_batch).float().mean().item()
print("Clean accuracy is {}.".format(clean_acc))

pred_adv = model(x_adv.to(DEVICE))[-1].max(dim=1)[1].detach().cpu()
adv_acc = (pred_adv == y_batch).float().mean().item()
print("Adversarial accuracy (CNN) is {}.".format(adv_acc))

Files already downloaded and verified
Files already downloaded and verified
Clean accuracy is 0.8759999871253967.
Adversarial accuracy (CNN) is 0.32199999690055847.


# DEFAULT CONFIGS

In [5]:
def get_default_config():
    """return the default config """
    return {
            "start_layer": 1,
            "n_class": 10,
            'aise_params' : [{"n_neighbors" : 10, 
                             "n_population" : 100, 
                             "hidden_layer": 1, 
                             "sampling_temperature": 1, 
                             "max_generation": 10, 
                             "mut_range": (.005, .015)}
            ]
}


get_default_config()

{'start_layer': 1,
 'n_class': 10,
 'aise_params': [{'n_neighbors': 10,
   'n_population': 100,
   'hidden_layer': 1,
   'sampling_temperature': 1,
   'max_generation': 10,
   'mut_range': (0.005, 0.015)}]}

# Neighbors

In [6]:
"""
neighbor number (2, 6, 10, 14, 18, 22, 26), 
"""

# params
NEIGHBOR_RANGE = [2 , 6, 10, 14, 18, 22, 26]
N_CLASS = 10

# result data structure
result_rows = []

for n in NEIGHBOR_RANGE:
    
    CONFIG = get_default_config() # intialize the default configs
    
    
    # update experiment params
    CONFIG['aise_params'][0]['n_neighbors'] = int(n)
    CONFIG['aise_params'][0]['n_population'] = int(n) * N_CLASS
        
    print(f"trainning with: n_neighbors = {int(n)}, n_population = {10 * int(n)} ")
    
    # build RAILS
    rails = RAILS(model, CONFIG, x_train, y_train)
    
    # compute accuracy on clean and poisoned data
    pred_rails = rails.predict(x_batch.to(DEVICE)).argmax(axis=1)
    acc_rails = (pred_rails == y_batch.numpy()).astype("float").mean().item()
    
    pred_adv_rails = rails.predict(x_adv.to(DEVICE)).argmax(axis=1)
    adv_acc_rails = (pred_adv_rails == y_batch.numpy()).astype("float").mean().item()
    
    # update results
    row = {
        'clean_acc' : acc_rails,
        'adv_acc' : adv_acc_rails,
        'acc_diff' : acc_rails - adv_acc_rails,
        'n_neighbors' : int(n),
        'n_population' : N_CLASS * int(n),
    }
    result_rows.append(row)
    
# print results
neighbors_results = pd.DataFrame(result_rows)
print(neighbors_results.to_latex(index=False))

trainning with: n_neighbors = 2, n_population = 20 
trainning with: n_neighbors = 6, n_population = 60 
trainning with: n_neighbors = 10, n_population = 100 
trainning with: n_neighbors = 14, n_population = 140 
trainning with: n_neighbors = 18, n_population = 180 
trainning with: n_neighbors = 22, n_population = 220 
trainning with: n_neighbors = 26, n_population = 260 
\begin{tabular}{rrrrr}
\toprule
 clean\_acc &  adv\_acc &  acc\_diff &  n\_neighbors &  n\_population \\
\midrule
     0.576 &    0.501 &     0.075 &            2 &            20 \\
     0.577 &    0.512 &     0.065 &            6 &            60 \\
     0.579 &    0.509 &     0.070 &           10 &           100 \\
     0.569 &    0.518 &     0.051 &           14 &           140 \\
     0.582 &    0.512 &     0.070 &           18 &           180 \\
     0.582 &    0.504 &     0.078 &           22 &           220 \\
     0.596 &    0.523 &     0.073 &           26 &           260 \\
\bottomrule
\end{tabular}



# Population

In [7]:
"""
population size (scale by 1, 2, 3, 4, 5 under neighbor number 10 and 2),
"""

# params
NEIGHBOR_RANGE = [2, 10]
POPULATION_RANGE = [1, 2, 3, 4, 5] # these are factors
N_CLASS = 10

# result data structure
result_rows = []

for n in NEIGHBOR_RANGE:
    for p in POPULATION_RANGE:
        
        CONFIG = get_default_config() # intialize the default configs
        
        
        # compute population size
        pop_size = N_CLASS * int(n) * int(p)
    
        # update experiment params
        CONFIG['aise_params'][0]['n_neighbors'] = int(n)
        CONFIG['aise_params'][0]['n_population'] = pop_size

        print(f"trainning with: n_neighbors = {int(n)}, n_population = {pop_size} ")

        # build RAILS
        rails = RAILS(model, CONFIG, x_train, y_train)

        # compute accuracy on clean and poisoned data
        pred_rails = rails.predict(x_batch.to(DEVICE)).argmax(axis=1)
        acc_rails = (pred_rails == y_batch.numpy()).astype("float").mean().item()

        pred_adv_rails = rails.predict(x_adv.to(DEVICE)).argmax(axis=1)
        adv_acc_rails = (pred_adv_rails == y_batch.numpy()).astype("float").mean().item()

        # update results
        row = {
            'clean_acc' : acc_rails,
            'adv_acc' : adv_acc_rails,
            'acc_diff' : acc_rails - adv_acc_rails,
            'n_neighbors' : int(n),
            'n_population' : pop_size,
        }
        result_rows.append(row)

        
# print results
population_results = pd.DataFrame(result_rows)
print(population_results.to_latex(index=False))

trainning with: n_neighbors = 2, n_population = 20 
trainning with: n_neighbors = 2, n_population = 40 
trainning with: n_neighbors = 2, n_population = 60 
trainning with: n_neighbors = 2, n_population = 80 
trainning with: n_neighbors = 2, n_population = 100 
trainning with: n_neighbors = 10, n_population = 100 
trainning with: n_neighbors = 10, n_population = 200 
trainning with: n_neighbors = 10, n_population = 300 
trainning with: n_neighbors = 10, n_population = 400 
trainning with: n_neighbors = 10, n_population = 500 
\begin{tabular}{rrrrr}
\toprule
 clean\_acc &  adv\_acc &  acc\_diff &  n\_neighbors &  n\_population \\
\midrule
     0.569 &    0.508 &     0.061 &            2 &            20 \\
     0.576 &    0.507 &     0.069 &            2 &            40 \\
     0.599 &    0.507 &     0.092 &            2 &            60 \\
     0.590 &    0.515 &     0.075 &            2 &            80 \\
     0.583 &    0.515 &     0.068 &            2 &           100 \\
     0.579 &   

# Mutation Range

In [ ]:
"""
mutation range upper bound (0.005, 0.01, 0.015, 0.02, 0.025, 0.03 
when lower bound is fixed to 0.002, nearest number 10)
"""

# params
NEIGHBOR_RANGE = [10]
MUTATION_LB = 0.002
MUTATION_UB = [0.005, 0.01, 0.015, 0.02, 0.025, 0.03]
N_CLASS = 10

# result data structure
result_rows = []

for n in NEIGHBOR_RANGE:
    for UB in MUTATION_UB:
        
        CONFIG = get_default_config() # intialize the default configs
        
        mutation_range = (MUTATION_LB, UB)
        
        # compute population size
        pop_size = N_CLASS * int(n) 
    
        # update experiment params
        CONFIG['aise_params'][0]['n_neighbors'] = int(n)
        CONFIG['aise_params'][0]['n_population'] = pop_size
        CONFIG['aise_params'][0]['mut_range'] = mutation_range

        print(f"trainning with: n_neighbors = {int(n)}, mutation_range = {mutation_range} ")

        # build RAILS
        rails = RAILS(model, CONFIG, x_train, y_train)

        # compute accuracy on clean and poisoned data
        pred_rails = rails.predict(x_batch.to(DEVICE)).argmax(axis=1)
        acc_rails = (pred_rails == y_batch.numpy()).astype("float").mean().item()

        pred_adv_rails = rails.predict(x_adv.to(DEVICE)).argmax(axis=1)
        adv_acc_rails = (pred_adv_rails == y_batch.numpy()).astype("float").mean().item()

        # update results
        row = {
            'clean_acc' : acc_rails,
            'adv_acc' : adv_acc_rails,
            'acc_diff' : acc_rails - adv_acc_rails,
            'n_neighbors' : int(n),
            'n_population' : pop_size,
            'mut_lb' : MUTATION_LB,
            'mut_ub' : UB
        }
        result_rows.append(row)

        
# print results
mutation_results = pd.DataFrame(result_rows)
print(mutation_results.to_latex(index=False))

trainning with: n_neighbors = 10, mutation_range = (0.002, 0.005) 
trainning with: n_neighbors = 10, mutation_range = (0.002, 0.01) 
trainning with: n_neighbors = 10, mutation_range = (0.002, 0.015) 
trainning with: n_neighbors = 10, mutation_range = (0.002, 0.02) 


# Mutation Probability

In [ ]:
"""
mutation probability (0.05, 0.1, 0.15, 0.2, 0.25, nearest neighbor 10).
"""

# params
NEIGHBOR_RANGE = [10]
PX_LB = 0.05
PX_UB = [0.05, 0.1, 0.15, 0.2, 0.25]
N_CLASS = 10

# result data structure
result_rows = []

for n in NEIGHBOR_RANGE:
    for UB in PX_UB:
        
        CONFIG = get_default_config() # intialize the default configs
        
        mutation_px = (PX_LB, UB)
        
        # compute population size
        pop_size = N_CLASS * int(n) 
    
        # update experiment params
        CONFIG['aise_params'][0]['n_neighbors'] = int(n)
        CONFIG['aise_params'][0]['n_population'] = pop_size
        CONFIG['aise_params'][0]['mut_prob'] = mutation_px

        print(f"trainning with: n_neighbors = {int(n)}, mutation_px = {mutation_px} ")

        # build RAILS
        rails = RAILS(model, CONFIG, x_train, y_train)

        # compute accuracy on clean and poisoned data
        pred_rails = rails.predict(x_batch.to(DEVICE)).argmax(axis=1)
        acc_rails = (pred_rails == y_batch.numpy()).astype("float").mean().item()

        pred_adv_rails = rails.predict(x_adv.to(DEVICE)).argmax(axis=1)
        adv_acc_rails = (pred_adv_rails == y_batch.numpy()).astype("float").mean().item()

        # update results
        row = {
            'clean_acc' : acc_rails,
            'adv_acc' : adv_acc_rails,
            'acc_diff' : acc_rails - adv_acc_rails,
            'n_neighbors' : int(n),
            'n_population' : pop_size,
            'mut_px_lb' : PX_LB,
            'mut_px_ub' : UB
        }
        result_rows.append(row)

        
# print results
probability_results = pd.DataFrame(result_rows)
print(probability_results.to_latex(index=False))

# Operator Type

In [ ]:
# params
NEIGHBOR_RANGE = [10]
POPULATION_RANGE = [1, 2, 5]
GENETIC_OPERATOR = ["crossover", "mutate"]
N_CLASS = 10

# result data structure
result_rows = []

for n in NEIGHBOR_RANGE:
    for p in POPULATION_RANGE:
        for op in GENETIC_OPERATOR:
            
            CONFIG = get_default_config() # intialize the default configs
        
        
            # compute population size
            pop_size = N_CLASS * int(n) * p

            # update experiment params
            CONFIG['aise_params'][0]['n_neighbors'] = int(n)
            CONFIG['aise_params'][0]['n_population'] = pop_size
            CONFIG['aise_params'][0]['genop_type'] = op

            print(f"trainning with: n_neighbors = {int(n)}, operator = {op} ")

            # build RAILS
            rails = RAILS(model, CONFIG, x_train, y_train)

            # compute accuracy on clean and poisoned data
            pred_rails = rails.predict(x_batch.to(DEVICE)).argmax(axis=1)
            acc_rails = (pred_rails == y_batch.numpy()).astype("float").mean().item()

            pred_adv_rails = rails.predict(x_adv.to(DEVICE)).argmax(axis=1)
            adv_acc_rails = (pred_adv_rails == y_batch.numpy()).astype("float").mean().item()

            # update results
            row = {
                'clean_acc' : acc_rails,
                'adv_acc' : adv_acc_rails,
                'acc_diff' : acc_rails - adv_acc_rails,
                'n_neighbors' : int(n),
                'n_population' : pop_size,
                'op' : op
            }
            result_rows.append(row)

        
# print results
operator_results = pd.DataFrame(result_rows)
print(operator_results.to_latex(index=False))

In [ ]:
CONFIG